<a href="https://colab.research.google.com/github/03aquamarine/YuJin-s/blob/main/0718Tue_%EA%B0%9C%EC%9D%B8%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B802_%EC%84%B1%EA%B7%A0%EA%B4%80%EB%8C%80_%EB%85%B8%EC%9C%A0%EC%A7%84_%EC%83%9D%EC%82%B0%EB%9D%BC%EC%9D%B8_%EB%B6%88%EB%9F%89%EC%A0%9C%ED%92%88_%EC%9E%90%EB%8F%99%ED%83%90%EC%A7%80%EC%8B%9C%EC%8A%A4%ED%85%9C_%EA%B0%9C%EB%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. data 불러오기
2. labeling
3. 전처리
4. 학습
5. predict

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Data 일괄 불러오기
import os , glob

good_data = glob.glob('/content/drive/MyDrive/grid/grid/train/good/*')
anomaly_data = glob.glob('/content/drive/MyDrive/grid/grid/test/*/*.png')

In [12]:
good_data
anomaly_data

['/content/drive/MyDrive/grid/grid/test/metal_contamination/007.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/004.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/010.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/008.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/001.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/003.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/002.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/006.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/000.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/009.png',
 '/content/drive/MyDrive/grid/grid/test/metal_contamination/005.png',
 '/content/drive/MyDrive/grid/grid/test/bent/001.png',
 '/content/drive/MyDrive/grid/grid/test/bent/000.png',
 '/content/drive/MyDrive/grid/grid/test/bent/011.png',
 '/content/drive/MyDrive/grid/grid/test/bent/009.png',
 '/content

In [13]:
import numpy as np
from PIL import Image

# 1. resize
# 2. type( RGB로)
# 3. 픽셀값 0~1 normalize (cf.생성쪽 -1 ~ 1)
# 4. numpy

x = []
y = []
data_folder = [good_data,anomaly_data]

for idx,folder in enumerate(data_folder):
  for file_path in folder:
      # 1. resize
      image = Image.open(file_path)
      image = image.resize((128,128))

      # 2. type to RGB
      image = image.convert("RGB")

      # 3. normalize pixel values (0~1)
      image = np.array(image) / 255

      # 4. store images as a numpy array
      x.append(image)
      y.append(idx)

x = np.array(x)
y = np.array(y)

In [8]:
## 모델 쌓기
import tensorflow as tf
from tensorflow.keras import layers, models

# CNN 모델 생성
model = models.Sequential()

# 첫 번째 Convolutional Layer, 32x32 input size, 32 filters, 3x3 filter size
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3),padding='same'))
model.add(layers.MaxPooling2D((3, 3), padding='same'))

# 두 번째 Convolutional Layer, 64 filters, 3x3 filter size
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
# 두 번째 Convolutional Layer, 64 filters, 3x3 filter size
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding='same'))

# 세 번째 Convolutional Layer, 64 filters, 3x3 filter size
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Fully Connected Layer로 변환
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))

# Output Layer, 10개의 클래스로 분류
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 5, 64)          36928     
                                                                 
 conv2d_3 (Conv2D)           (None, 5, 5, 64)          3

In [9]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [10]:
x_train.shape
y_train.shape
x_train = x_train / 255
x_test = x_test / 255
x_train[0]

array([[[0.23137255, 0.24313725, 0.24705882],
        [0.16862745, 0.18039216, 0.17647059],
        [0.19607843, 0.18823529, 0.16862745],
        ...,
        [0.61960784, 0.51764706, 0.42352941],
        [0.59607843, 0.49019608, 0.4       ],
        [0.58039216, 0.48627451, 0.40392157]],

       [[0.0627451 , 0.07843137, 0.07843137],
        [0.        , 0.        , 0.        ],
        [0.07058824, 0.03137255, 0.        ],
        ...,
        [0.48235294, 0.34509804, 0.21568627],
        [0.46666667, 0.3254902 , 0.19607843],
        [0.47843137, 0.34117647, 0.22352941]],

       [[0.09803922, 0.09411765, 0.08235294],
        [0.0627451 , 0.02745098, 0.        ],
        [0.19215686, 0.10588235, 0.03137255],
        ...,
        [0.4627451 , 0.32941176, 0.19607843],
        [0.47058824, 0.32941176, 0.19607843],
        [0.42745098, 0.28627451, 0.16470588]],

       ...,

       [[0.81568627, 0.66666667, 0.37647059],
        [0.78823529, 0.6       , 0.13333333],
        [0.77647059, 0

In [15]:
model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=10)

Epoch 1/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.3287 - accuracy: 0.8838 - val_loss: 1.2184 - val_accuracy: 0.7020
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.3115 - accuracy: 0.8906 - val_loss: 1.2315 - val_accuracy: 0.7052
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.3081 - accuracy: 0.8913 - val_loss: 1.2013 - val_accuracy: 0.7023
Epoch 4/10
1563/1563 [==============================] - 10s 7ms/step - loss: 0.2977 - accuracy: 0.8948 - val_loss: 1.2079 - val_accuracy: 0.7064
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2897 - accuracy: 0.8987 - val_loss: 1.2213 - val_accuracy: 0.7004
Epoch 6/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.2778 - accuracy: 0.9021 - val_loss: 1.3285 - val_accuracy: 0.7075
Epoch 7/10
1563/1563 [==============================] - 10s 7ms/step - loss: 0.2691 - accuracy: 0.9063 - val_loss: 1.3749 - val_ac